In [2]:
import yfinance as yf
import os 
import pandas as pd 
import numpy as np



In [3]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]
sp500_tickers = sp500_table['Symbol'].tolist()


In [ ]:

ticker = "AOS"

data = yf.download(ticker, period="max")
data.append('stockData.csv', index=False)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
